<a href="https://colab.research.google.com/github/salonijaitly/ML_Sem_V/blob/master/ML_05_J070_140820/ML_05_J070_140820.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regression and Classification using linear models


In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn import linear_model
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

#**FUEL CONSUMPTION**

In [ ]:
!wget -O FuelConsumption.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/FuelConsumptionCo2.csv

--2020-08-29 01:18:10--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/FuelConsumptionCo2.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72629 (71K) [text/csv]
Saving to: ‘FuelConsumption.csv’

FuelConsumption.csv 100%[===================>]  70.93K   420KB/s    in 0.2s    

2020-08-29 01:18:11 (420 KB/s) - ‘FuelConsumption.csv’ saved [72629/72629]



In [ ]:
fuelcon = pd.read_csv('FuelConsumption.csv')
fuelcon.head()

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


In [ ]:
fuelcon.columns

Index(['MODELYEAR', 'MAKE', 'MODEL', 'VEHICLECLASS', 'ENGINESIZE', 'CYLINDERS',
       'TRANSMISSION', 'FUELTYPE', 'FUELCONSUMPTION_CITY',
       'FUELCONSUMPTION_HWY', 'FUELCONSUMPTION_COMB',
       'FUELCONSUMPTION_COMB_MPG', 'CO2EMISSIONS'],
      dtype='object')

In [ ]:
print(fuelcon.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MODELYEAR                 1067 non-null   int64  
 1   MAKE                      1067 non-null   object 
 2   MODEL                     1067 non-null   object 
 3   VEHICLECLASS              1067 non-null   object 
 4   ENGINESIZE                1067 non-null   float64
 5   CYLINDERS                 1067 non-null   int64  
 6   TRANSMISSION              1067 non-null   object 
 7   FUELTYPE                  1067 non-null   object 
 8   FUELCONSUMPTION_CITY      1067 non-null   float64
 9   FUELCONSUMPTION_HWY       1067 non-null   float64
 10  FUELCONSUMPTION_COMB      1067 non-null   float64
 11  FUELCONSUMPTION_COMB_MPG  1067 non-null   int64  
 12  CO2EMISSIONS              1067 non-null   int64  
dtypes: float64(4), int64(4), object(5)
memory usage: 108.5+ KB
None

In [ ]:
columns = fuelcon.columns

In [ ]:
for col in columns:
  print(col)
  print(fuelcon[col].nunique())

MODELYEAR
1
MAKE
39
MODEL
663
VEHICLECLASS
16
ENGINESIZE
45
CYLINDERS
7
TRANSMISSION
22
FUELTYPE
4
FUELCONSUMPTION_CITY
167
FUELCONSUMPTION_HWY
118
FUELCONSUMPTION_COMB
148
FUELCONSUMPTION_COMB_MPG
43
CO2EMISSIONS
159


In [ ]:
fuelcon.drop('MODELYEAR', axis = 1, inplace = True)
fuelcon.drop('MODEL', axis = 1, inplace = True)
fuelcon.drop('MAKE', axis = 1, inplace = True)
fuelcon.head()

,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


In [ ]:
fuelcon = pd.get_dummies(fuelcon)

In [ ]:
fuelcon.dtypes

ENGINESIZE                               float64
CYLINDERS                                  int64
FUELCONSUMPTION_CITY                     float64
FUELCONSUMPTION_HWY                      float64
FUELCONSUMPTION_COMB                     float64
FUELCONSUMPTION_COMB_MPG                   int64
CO2EMISSIONS                               int64
VEHICLECLASS_COMPACT                       uint8
VEHICLECLASS_FULL-SIZE                     uint8
VEHICLECLASS_MID-SIZE                      uint8
VEHICLECLASS_MINICOMPACT                   uint8
VEHICLECLASS_MINIVAN                       uint8
VEHICLECLASS_PICKUP TRUCK - SMALL          uint8
VEHICLECLASS_PICKUP TRUCK - STANDARD       uint8
VEHICLECLASS_SPECIAL PURPOSE VEHICLE       uint8
VEHICLECLASS_STATION WAGON - MID-SIZE      uint8
VEHICLECLASS_STATION WAGON - SMALL         uint8
VEHICLECLASS_SUBCOMPACT                    uint8
VEHICLECLASS_SUV - SMALL                   uint8
VEHICLECLASS_SUV - STANDARD                uint8
VEHICLECLASS_TWO-SEA

In [ ]:
fuelcon.head()

,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS,VEHICLECLASS_COMPACT,VEHICLECLASS_FULL-SIZE,VEHICLECLASS_MID-SIZE,VEHICLECLASS_MINICOMPACT,VEHICLECLASS_MINIVAN,VEHICLECLASS_PICKUP TRUCK - SMALL,VEHICLECLASS_PICKUP TRUCK - STANDARD,VEHICLECLASS_SPECIAL PURPOSE VEHICLE,VEHICLECLASS_STATION WAGON - MID-SIZE,VEHICLECLASS_STATION WAGON - SMALL,VEHICLECLASS_SUBCOMPACT,VEHICLECLASS_SUV - SMALL,VEHICLECLASS_SUV - STANDARD,VEHICLECLASS_TWO-SEATER,VEHICLECLASS_VAN - CARGO,VEHICLECLASS_VAN - PASSENGER,TRANSMISSION_A4,TRANSMISSION_A5,TRANSMISSION_A6,TRANSMISSION_A7,TRANSMISSION_A8,TRANSMISSION_A9,TRANSMISSION_AM5,TRANSMISSION_AM6,TRANSMISSION_AM7,TRANSMISSION_AS4,TRANSMISSION_AS5,TRANSMISSION_AS6,TRANSMISSION_AS7,TRANSMISSION_AS8,TRANSMISSION_AS9,TRANSMISSION_AV,TRANSMISSION_AV6,TRANSMISSION_AV7,TRANSMISSION_AV8,TRANSMISSION_M5,TRANSMISSION_M6,TRANSMISSION_M7,FUELTYPE_D,FUELTYPE_E,FUELTYPE_X,FUELTYPE_Z
0,2.0,4,9.9,6.7,8.5,33,196,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2.4,4,11.2,7.7,9.6,29,221,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1.5,4,6.0,5.8,5.9,48,136,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,3.5,6,12.7,9.1,11.1,25,255,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,3.5,6,12.1,8.7,10.6,27,244,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
x = fuelcon.drop('CO2EMISSIONS', axis = 1)
y = fuelcon['CO2EMISSIONS']
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.6)

In [ ]:
linear = linear_model.LinearRegression()
ridge = linear_model.Ridge()
lasso= linear_model.Lasso()
elastic = linear_model.ElasticNet()
lasso_lars = linear_model.LassoLars()
bayes_ridge = linear_model.BayesianRidge()
logistics = linear_model.LogisticRegression()
sgd = linear_model.SGDClassifier()
passagg = linear_model.PassiveAggressiveClassifier()
ridgecv = linear_model.RidgeClassifierCV()
ridgeclass = linear_model.RidgeClassifier()
models_churn = [linear, ridge, lasso, elastic, lasso_lars, bayes_ridge]

In [ ]:
def get_cv_scores(model):
    scores = cross_val_score(model, x_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    print('CV Mean: ', np.mean(scores))
    print('STD: ', np.std(scores))
    print('\n')

In [ ]:
for i in models_churn:
    print(i)
    get_cv_scores(i)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
CV Mean:  -6.5332567116288045
STD:  0.5331672077444968


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
CV Mean:  -6.540853607239823
STD:  0.4355116030713686


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
CV Mean:  -9.660323485757502
STD:  1.1871565189447755


ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
CV Mean:  -21.10895287242052
STD:  1.7012917477255731


LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
          fit_path=True, max_iter=500, normalize=True, pos

In [ ]:
alpha = [0.5, 1, 1.5, 0.01, 2.5, 0.0001, 10, 100, 0.35]
solver = ['auto', 'svd']
param_grid = dict(alpha = alpha, solver = solver)

In [ ]:
gridsearch = GridSearchCV(estimator = ridge, param_grid = param_grid, scoring = 'r2', cv = 10)
grid_result = gridsearch.fit(x_train, y_train)

In [ ]:
rand_srch = RandomizedSearchCV(estimator = ridge, param_distributions = param_grid, scoring='r2', cv = 10)
rnds_result = rand_srch.fit(x_train, y_train)

In [ ]:
print(grid_result.best_params_)
print(rnds_result.best_params_)

{'alpha': 0.01, 'solver': 'auto'}
{'solver': 'auto', 'alpha': 0.01}


In [ ]:
print(grid_result.best_score_)
print(rnds_result.best_score_)

0.9902179947695833
0.9902179947695833


In [ ]:
ridge=linear_model.Ridge(alpha=0.05,solver='auto')
ridge.fit(x_train, y_train)

Ridge(alpha=0.05, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
y_pred = ridge.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score,r2_score

In [ ]:
r2_score(y_test,y_pred)

0.9927466404630325

#**IRIS DATASET**

In [ ]:
iris_data = load_iris()

In [ ]:
data = pd.DataFrame(data= np.c_[iris_data['data'], iris_data['target']],columns= iris_data['feature_names'] + ['target'])
encoder=OneHotEncoder(sparse=False)
print(data)

     sepal length (cm)  sepal width (cm)  ...  petal width (cm)  target
0                  5.1               3.5  ...               0.2     0.0
1                  4.9               3.0  ...               0.2     0.0
2                  4.7               3.2  ...               0.2     0.0
3                  4.6               3.1  ...               0.2     0.0
4                  5.0               3.6  ...               0.2     0.0
..                 ...               ...  ...               ...     ...
145                6.7               3.0  ...               2.3     2.0
146                6.3               2.5  ...               1.9     2.0
147                6.5               3.0  ...               2.0     2.0
148                6.2               3.4  ...               2.3     2.0
149                5.9               3.0  ...               1.8     2.0

[150 rows x 5 columns]


In [ ]:
data.dtypes

sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
target               float64
dtype: object

In [ ]:
x=data1.drop('target',axis=1)
y=data1['target']
X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistics = linear_model.LogisticRegression()
sgd = linear_model.SGDClassifier()
passagg = linear_model.PassiveAggressiveClassifier()
ridgecv = linear_model.RidgeClassifierCV()
ridgeclass = linear_model.RidgeClassifier()
models_log = [logistics, sgd, passagg, ridgecv, ridgeclass]

In [ ]:
def get_cv_scores(model):
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    print('CV Mean: ', np.mean(scores))
    print('STD: ', np.std(scores))
    print('\n')

In [ ]:
for i in models_log:
    print(i)
    get_cv_scores(i)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
CV Mean:  0.95
STD:  0.0311804782231162


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
CV Mean:  0.8833333333333332
STD:  0.07168604389202189


PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
            

In [ ]:
solver = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
param_grid = dict(solver = solver)
gridsearch = GridSearchCV(estimator = logistics, param_grid = param_grid, scoring = 'r2', cv = 10)
grid_result = gridsearch.fit(X_train, y_train)

In [ ]:
rand_srch = RandomizedSearchCV(estimator = logistics, param_distributions = param_grid, scoring='r2', cv = 10)
rnds_result = rand_srch.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


In [ ]:
print(grid_result.best_params_)
print(rnds_result.best_params_)

{'solver': 'newton-cg'}
{'solver': 'newton-cg'}


In [ ]:
print(grid_result.best_score_)
print(rnds_result.best_score_)

0.923913043478261
0.923913043478261


In [ ]:
log_reg = LogisticRegression(solver='newton-cg')
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
r2_score(y_test,y_pred)

1.0

In [ ]:
accuracy_score(y_test,y_pred)

1.0